In [22]:
import re

In [23]:
test_string = """Monkey 0:
  Starting items: 79, 98
  Operation: new = old * 19
  Test: divisible by 23
    If true: throw to monkey 2
    If false: throw to monkey 3

Monkey 1:
  Starting items: 54, 65, 75, 74
  Operation: new = old + 6
  Test: divisible by 19
    If true: throw to monkey 2
    If false: throw to monkey 0

Monkey 2:
  Starting items: 79, 60, 97
  Operation: new = old * old
  Test: divisible by 13
    If true: throw to monkey 1
    If false: throw to monkey 3

Monkey 3:
  Starting items: 74
  Operation: new = old + 3
  Test: divisible by 17
    If true: throw to monkey 0
    If false: throw to monkey 1"""

In [24]:

from dataclasses import dataclass


@dataclass
class Monkey:
    id: str = None,
    items: list = None,
    throw_to: dict = None,
    op: str = None,
    operand: str = None
    modulus: int = None
    others: dict = None
    inspect_count: int = 0
    
    def check(self):
        return self.id is not None and \
               self.items is not None and \
               self.throw_to is not None and \
               self.op is not None and \
               self.operand is not None and \
               self.modulus is not None
               
    def inspect(self, item):
        newitem = {}
        if not isinstance(item, dict):
            newitem = { m.id: (item % m.modulus) for m in self.others.values()}
            item = {m.id: item for m in self.others.values()}
        else:
            for k in item.keys():
                newitem[k] = item[k] % self.others[k].modulus
        
        for k in newitem.keys():
            mm = self.others[k].modulus
            if self.operand == "old":
                if self.op == "+":
                    newitem[k] = (newitem[k] + newitem[k]) % mm 
                if self.op == "*":
                    newitem[k] = (newitem[k] * newitem[k]) % mm
            else:
                operand = int(self.operand) % mm
                if self.op == "+":
                    newitem[k] = (newitem[k] + operand) % mm
                if self.op == "*":
                    newitem[k] = (newitem[k] * operand) % mm
        return newitem
            
            
    def inspect_all(self):
        self.inspect_count += len(self.items)
        for i in range(len(self.items)):
            # print(f"Monkey {self.id} inspects {self.items[i]}, new worry {self.inspect(self.items[i]) // 3}")
            self.items[i] = self.inspect(self.items[i]) 
    
    def throw_item(self, item):
        other = self.others[self.throw_to[item[self.id] % self.modulus == 0]]
        #print(f"Monkey {self.id} throws {item} at Monkey {other.id}")
        other.items.append(item)
        
    def monkey_round(self):
        self.inspect_all()
        self.items.reverse()
        while len(self.items) > 0:
            self.throw_item(self.items.pop())
            
               


In [25]:
def parse(input: str) -> dict[Monkey]:
    monkey_buffer = None
    monkeys = {}
    for line in input.splitlines():
        line = line.strip()
        if len(line) == 0: continue
        if line.startswith("Monkey"):
            if monkey_buffer is not None: 
                if monkey_buffer.check():
                    monkeys[monkey_buffer.id] = monkey_buffer
                else:
                    raise ValueError(f"Malformed Monkey {monkey_buffer.id}")
            monkey_buffer = Monkey()
            monkey_buffer.id = int(re.search("Monkey ([\d]+):", line).groups()[0])
            monkey_buffer.items = []
            monkey_buffer.throw_to = {}
            monkey_buffer.others = monkeys
            continue
        if line.startswith("Starting items"):
            monkey_buffer.items = [int(x) for x in (re.search("Starting items: (.+)", line).groups()[0].split(","))]
            continue
        if line.startswith("Operation:"):
            op, operand = re.search("Operation: new = old (.) (.+)", line).groups()
            monkey_buffer.op = op
            monkey_buffer.operand = operand
            continue
        if line.startswith("Test"):
            monkey_buffer.modulus = int(re.search("Test: divisible by (\d+)", line).groups()[0])
            continue
        if line.startswith("If true"):
            monkey_buffer.throw_to[True] = int(re.search("If true: throw to monkey (\d+)", line).groups()[0])
            continue
        if line.startswith("If false"):
            monkey_buffer.throw_to[False] = int(re.search("If false: throw to monkey (\d+)", line).groups()[0])
            continue
        raise ValueError(f"Bad input: {line}")
    if monkey_buffer is not None: 
        if monkey_buffer.check():
            monkeys[monkey_buffer.id] = monkey_buffer
        else:
            raise ValueError(f"Malformed Monkey {monkey_buffer.id}")
    return monkeys
        

        

In [34]:
monkeys = parse(open("../inputs/11.txt").read())
#monkeys =parse(test_string)
for i in range(10000):
    #print(f"round {i}")
    for monkey in monkeys.values():
        monkey.monkey_round()

activity = sorted([monkey.inspect_count for monkey in monkeys.values()], reverse=True)
activity[0] * activity[1]

54832778815

In [32]:
activity

[52166, 47830, 1938, 52013]

In [28]:
monkeys[0]

Monkey(id=0, items=[{0: 18, 1: 9, 2: 1, 3: 5}, {0: 5, 1: 9, 2: 4, 3: 8}, {0: 2, 1: 9, 2: 7, 3: 13}, {0: 19, 1: 9, 2: 0, 3: 8}, {0: 9, 1: 9, 2: 6, 3: 13}], throw_to={True: 2, False: 3}, op='*', operand='19', modulus=23, others={0: ..., 1: Monkey(id=1, items=[{0: 20, 1: 3, 2: 0, 3: 16}, {0: 11, 1: 3, 2: 1, 3: 5}, {0: 10, 1: 3, 2: 11, 3: 10}, {0: 6, 1: 3, 2: 11, 3: 4}, {0: 16, 1: 3, 2: 8, 3: 4}], throw_to={True: 2, False: 0}, op='+', operand='6', modulus=19, others={...}, inspect_count=97), 2: Monkey(id=2, items=[], throw_to={True: 1, False: 3}, op='*', operand='old', modulus=13, others={...}, inspect_count=8), 3: Monkey(id=3, items=[], throw_to={True: 0, False: 1}, op='+', operand='3', modulus=17, others={...}, inspect_count=103)}, inspect_count=99)